In [2]:
from pathlib import Path
from IPython.display import Markdown, HTML

pages_root = Path(".")
print(f'{pages_root.absolute() = }')
# readmes = pages_root.glob("**/README.md")

# Add links to all htmls and markdown files at the bottom of the README.md

md = ""

with open(pages_root.joinpath("README.md"), "a+") as f:
# with open(readme, "a+") as f:
    # f.write("\n## Sitemap Markdowns\n\n")
    md += "\n## Sitemap Markdowns\n\n"
    mds = sorted([p.relative_to(pages_root).as_posix() for p in pages_root.glob("**/*.md")])
    for link in (f'- [{p}]({p})\n' for p in mds):
        # f.write(link)
        md += f'\n{link}'
    # f.write("\n## Sitemap HTMLs\n\n")
    md += "\n## Sitemap HTMLs\n\n"
    htmls = [p for p in pages_root.glob("**/*.html")]
    htmls = sorted(htmls + list(set([x.parent for x in htmls])))
    base_dir = ""
    for link in htmls:
        if link.is_dir():
            base_dir = link
            # f.write(f'- {link.relative_to(pages_root).as_posix()}\n')
            md+= f'- {link.relative_to(pages_root).as_posix()}\n'
        else:
            link_text = link.relative_to(base_dir).stem
            for p in link.parent.glob("*"):
                if link.stem == p.stem and link.suffix != p.suffix:
                    link_text = p.relative_to(base_dir).name
            # f.write(f'\t- [{link_text}]({link.relative_to(pages_root)})\n')
            md += f'\t- [{link_text}]({link.relative_to(pages_root)})\n'

display(Markdown(md))

pages_root.absolute() = PosixPath('/workspaces/QuantumAnnealingPlayground')


In [5]:
get_markdown_files(pages_root)
# get_html_files_and_dirs(pages_root)

['README.md', 'choosing_boxes_problem/README.md']

In [39]:
p2 = Path(".")
p2 = Path("choosing_boxes_problem")
list(p2.glob("**/*.md"))
list([p.relative_to(p2) for p in p2.glob("**/*.md")])

[PosixPath('README.md')]

In [44]:
import itertools
from pathlib import Path
from pprint import pprint
from IPython.display import Markdown, HTML


def get_markdown_files(root_dir):
    return sorted([p.relative_to(root_dir) for p in root_dir.glob("**/*.md")])

def get_html_files_and_dirs(pages_root):
    html_files = list(pages_root.glob("**/*.html"))
    html_dirs = list(set([x.parent for x in html_files]))
    return sorted(html_files + html_dirs)

def get_html_files_and_dirs_grouped(pages_root):
    html_files = sorted(pages_root.glob("**/*.html"))
    # html_files = [p.relative_to(p.parent.parent) for p in html_files]
    # html_files = [p.name for p in html_files]
    # html_files = sorted([p.resolve().relative_to(pages_root.resolve()) for p in pages_root.glob("**/*.html")])
    html_tree = sorted([g, sorted(fs)] for g, fs in itertools.groupby(html_files, key=lambda x: x.parent))
    # html_tree = sorted([g, sorted([Path(p.name) for p in fs])] for g, fs in itertools.groupby(html_files, key=lambda x: x.parent))
    return html_tree

def make_md_link(url, text=None, list=False, indent=0):
    return ("\t" * indent) + ("- " if (list | indent) else "") + f"[{text or url}]({url})"

def make_md_link_list(urls, level=0, root=Path("."), remove_suffix="", orig_suffix=False):
    links = []
    for url in urls:
        text = None
        if remove_suffix:
            text = url.name.removesuffix(remove_suffix)
        if orig_suffix:
            for p in url.parent.glob(f'{url.stem}.*'):
                # if url.stem == p.stem and url.suffix != p.suffix:
                if url.suffix != p.suffix:
                    # text = p.relative_to(base_dir).name
                    text = p.name
                    # break
        links.append(make_md_link(url.relative_to(root), text=text, list=True, indent=level))
    return "\n".join(links)
    return "\n".join(make_md_link(
        url,
        # text=None if not remove_suffix else url.name.removesuffix(remove_suffix),
        text=None if not remove_suffix and not orig_suffix else f'{url.name.removesuffix("")}',
        list=True,
        indent=level) for url in urls)

def _generate_md_file_tree(dirs, pages_root=Path("."), remove_suffix=".html"):
    links = []
    for item in sorted(dirs):
        if item.is_dir():
            links.append(f'- {item.relative_to(pages_root).as_posix()}')
        else:
            link_text = item.relative_to(item.parent).name.removesuffix(remove_suffix)
            links.append(make_md_link(item.relative_to(pages_root), link_text, indent=1))
    return "\n".join(links)

def generate_md_file_tree(dirs, pages_root=Path("."), remove_suffix=".html"):
    links = []
    for parent, files in dirs:
        links.append(f'- {parent.relative_to(pages_root).as_posix()}')
        links.append(make_md_link_list(files, level=1, root=pages_root, remove_suffix=remove_suffix, orig_suffix=True))
    return "\n".join(links)

def update_readme_with_links(root_dir):
    md_files = get_markdown_files(root_dir)
    # html_items = get_html_files_and_dirs(root_dir)
    html_items = get_html_files_and_dirs_grouped(root_dir)
    # with open(root_dir.joinpath("README.md"), "a+") as f:
    #     f.write("\n## Sitemap Markdowns\n\n")
    #     f.write(make_md_link_list(md_files))
    #     f.write("\n\n## Sitemap HTMLs\n\n")
    #     f.write(generate_md_file_tree(html_items, pages_root=root_dir))
    md = "\n## Sitemap Markdowns\n\n"
    md += make_md_link_list(md_files)
    md += "\n\n## Sitemap HTMLs\n\n"
    md += generate_md_file_tree(html_items, pages_root=root_dir)
    return md

def insert_toc_in_readmes(root_dir):
    for readme in root_dir.glob("**/README.md"):
        # update_readme_with_links(readme.parent)
        # display(Markdown(update_readme_with_links(readme.parent)))
        print(update_readme_with_links(readme.parent))

pages_root2 = Path(".")
# update_readme_with_links(pages_root)
# display(Markdown(update_readme_with_links(pages_root)))
insert_toc_in_readmes(pages_root2)


## Sitemap Markdowns

- [README.md](README.md)
- [choosing_boxes_problem/README.md](choosing_boxes_problem/README.md)

## Sitemap HTMLs

- .
	- [playground.ipynb](playground.html)
- choosing_boxes_problem
	- [choosing_boxes.ipynb](choosing_boxes_problem/choosing_boxes.html)
- choosing_boxes_problem/results
	- [choosing-boxes-2-eigenspectrums-dt50](choosing_boxes_problem/results/choosing-boxes-2-eigenspectrums-dt50.html)
	- [choosing-boxes-2-eigenvectors-dt50](choosing_boxes_problem/results/choosing-boxes-2-eigenvectors-dt50.html)
	- [choosing-boxes-4-eigenspectrums-dt50](choosing_boxes_problem/results/choosing-boxes-4-eigenspectrums-dt50.html)
	- [choosing-boxes-4-eigenvectors-dt50](choosing_boxes_problem/results/choosing-boxes-4-eigenvectors-dt50.html)
	- [eigenspectrums-choose-2of2_γ5-80_Δt100_sched-approx](choosing_boxes_problem/results/eigenspectrums-choose-2of2_γ5-80_Δt100_sched-approx.html)
	- [eigenspectrums-choose-2of2_γ5-80_Δt100_sched-similar](choosing_boxes_problem/results/

## Mercury

In [53]:
import mercury

# https://runmercury.com/docs/input-widgets/app/
app = mercury.App(
    # title="",
    # description="",
    show_code=True,
    # show_prompt=False,
    continuous_update=False,
    static_notebook=True,
    # show_sidebar=True,
    # full_screen=True,
    # allow_download=True,
    # allow_share=False,
    # stop_on_error=False
)

mercury.App